**SERVICE UNAVAILABLE**

**This notebook is no longer supported. Please use [ColabFold](https://github.com/sokrypton/ColabFold) (for AlphaFold 2) or [AlphaFold Server](https://alphafoldserver.com/) (for AlphaFold 3).**
